In [10]:
import re
import jieba
import os
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathspec import pattern
from tornado.web import url
from wordcloud import WordCloud
from os import path
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis.sklearn
import pyLDAvis
import joblib

In [3]:
data = pd.read_excel('clean_data.xlsx')
data

,target,userId,sellerId,timestamp,comment
0,1,127,1708,2019-10-01 00:57:45,订餐 半小时 不见 餐 催 挂 电话
1,1,1661,886,2019-10-01 00:42:07,太 不靠 谱 漏发 订单 菜 送错 早 送 小时
2,0,92758,10612,2019-10-01 00:51:18,超级 超级 喜欢 吃 每次 好开心 口味 建议 手套 戴 几双 吃 完 形象...
3,0,1639,1699,2019-10-03 00:01:56,谢谢 快递 哥哥 准时 辛苦
4,1,241839,36556,2019-10-01 00:07:56,差得 餐厅 上菜 慢 贵 餐厅 饿死 那种 面条 味道 一碗 ...
...,...,...,...,...,...
17948,0,34025,24833,2020-09-25 23:15:36,感觉 好像 发挥 稳定 几此 有好有坏 吃 日料 关注 三文鱼 中等 不...
17949,1,656,901,2020-09-25 23:17:49,订 九点 送到 十二点
17950,1,612,743,2020-09-27 23:36:59,差评 辣白菜 五花肉 石锅 拌 饭 超级 难吃 五花肉 硬 难吃 感觉 新鲜 ...
17951,1,146739,15687,2020-09-27 23:53:15,贵 好吃 老婆 几次 非要 一吃 真 难吃 蒸 没错 炒菜 油腻 菜 好...


In [4]:
data = data[data.target==1]
data.to_excel('./Q4/group_Negativity_data.xlsx')
data

,target,userId,sellerId,timestamp,comment
0,1,127,1708,2019-10-01 00:57:45,订餐 半小时 不见 餐 催 挂 电话
1,1,1661,886,2019-10-01 00:42:07,太 不靠 谱 漏发 订单 菜 送错 早 送 小时
4,1,241839,36556,2019-10-01 00:07:56,差得 餐厅 上菜 慢 贵 餐厅 饿死 那种 面条 味道 一碗 ...
6,1,1519,1587,2019-10-01 00:14:29,送 两个 小时 解释 真的 打电话 问 醉
7,1,868,643,2019-10-03 00:08:53,吃 有股 异味 闻
...,...,...,...,...,...
17943,1,1687,851,2020-09-25 23:30:28,标注 奶油 还放 奶油 饮料 吸管
17949,1,656,901,2020-09-25 23:17:49,订 九点 送到 十二点
17950,1,612,743,2020-09-27 23:36:59,差评 辣白菜 五花肉 石锅 拌 饭 超级 难吃 五花肉 硬 难吃 感觉 新鲜 ...
17951,1,146739,15687,2020-09-27 23:53:15,贵 好吃 老婆 几次 非要 一吃 真 难吃 蒸 没错 炒菜 油腻 菜 好...


In [5]:
counts = Counter(data['sellerId'])
print(max(counts.keys(), key=counts.get))
print(counts.most_common(1))

971
[(971, 44)]


In [6]:
data = data[data.sellerId==max(counts.keys(), key=counts.get)]
data.to_excel('./Q4/group_max_Negativity_data.xlsx')
data

,target,userId,sellerId,timestamp,comment
42,1,212,971,2019-10-01 00:33:06,送餐 人员 态度恶劣 超过 一个半 小时 送来 骂人 外卖 从没 见过 送餐 人员
208,1,1075,971,2019-10-05 00:15:34,超过 预计 时间 太慢 好吃 凉
219,1,1406,971,2019-10-06 00:47:06,量少 点
621,1,1034,971,2019-10-13 01:36:14,送货 时间 太 长
623,1,1591,971,2019-10-13 01:18:57,汤 依旧 难以 下咽
1005,1,1123,971,2019-10-21 02:42:56,特别 少 菜 半盒 多一点
1149,1,1634,971,2019-10-24 03:21:14,评分 低 服务态度 接线员 工 态度 太 差 建议 订餐 电话 催单
1177,1,33,971,2019-10-25 03:53:13,太慢
1581,1,1768,971,2019-11-02 05:52:12,真的 送 饮料 洒 袋子 湿
1631,1,1827,971,2019-11-03 06:10:15,米线 全撒 汤 米线 漏出来 巨 恶心


In [7]:
cut_text=''
for i in data[data['target']==1]['comment']:
    cut_text=cut_text+str(i)
cut_textlist=cut_text.split()
sorted(Counter(cut_textlist).items(),key=lambda x:x[1],reverse=True)[:10]

[('送', 7),
 ('小时', 5),
 ('送来', 3),
 ('时间', 3),
 ('太', 3),
 ('少', 3),
 ('菜', 3),
 ('态度', 3),
 ('筷子', 3),
 ('吃', 3)]

In [8]:
plt.figure(figsize=(20,20),dpi=800)
cloud=WordCloud(font_path="./font.ttf",
                background_color='white',
                max_words=1000,
                scale=4)
word_cloud=cloud.generate_from_text(cut_text)
plt.imshow(word_cloud)
plt.axis('off')
plt.savefig('./Q4/max_sellerId_Negativity.jpg')
plt.show()

E:\anaconda\lib\site-packages\wordcloud\wordcloud.py:508: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  box_size = draw.textsize(word, font=transposed_font)
E:\anaconda\lib\site-packages\wordcloud\wordcloud.py:508: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  box_size = draw.textsize(word, font=transposed_font)
E:\anaconda\lib\site-packages\wordcloud\wordcloud.py:519: DeprecationWarning: ROTATE_90 is deprecated and will be removed in Pillow 10 (2023-07-01). Use Transpose.ROTATE_90 instead.
  orientation = (Image.ROTATE_90 if orientation is None else
E:\anaconda\lib\site-packages\wordcloud\wordcloud.py:508: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  box_size = draw.textsize(word, font=transposed_font)
E:\anaconda\lib\site-packages\wordcloud\

E:\anaconda\lib\site-packages\wordcloud\wordcloud.py:508: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  box_size = draw.textsize(word, font=transposed_font)
E:\anaconda\lib\site-packages\wordcloud\wordcloud.py:519: DeprecationWarning: ROTATE_90 is deprecated and will be removed in Pillow 10 (2023-07-01). Use Transpose.ROTATE_90 instead.
  orientation = (Image.ROTATE_90 if orientation is None else
E:\anaconda\lib\site-packages\wordcloud\wordcloud.py:508: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  box_size = draw.textsize(word, font=transposed_font)
E:\anaconda\lib\site-packages\wordcloud\wordcloud.py:508: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  box_size = draw.textsize(word, font=transposed_font)
E:\anaconda\lib\site-packages\wordcloud\

E:\anaconda\lib\site-packages\wordcloud\wordcloud.py:508: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  box_size = draw.textsize(word, font=transposed_font)
E:\anaconda\lib\site-packages\wordcloud\wordcloud.py:499: DeprecationWarning: ROTATE_90 is deprecated and will be removed in Pillow 10 (2023-07-01). Use Transpose.ROTATE_90 instead.
  orientation = Image.ROTATE_90
E:\anaconda\lib\site-packages\wordcloud\wordcloud.py:508: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  box_size = draw.textsize(word, font=transposed_font)
E:\anaconda\lib\site-packages\wordcloud\wordcloud.py:508: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  box_size = draw.textsize(word, font=transposed_font)
E:\anaconda\lib\site-packages\wordcloud\wordcloud.py:499: Deprecation

In [14]:
source_path = './Q4/group_max_Negativity_data.xlsx'

document_column_name = 'comment'

top_words_csv_path = './Q4/top-topic-words.xlsx'

predict_topic_csv_path = './Q4/document-distribution.xlsx'

html_path = './Q4/document-lda-visualization-top.html'

n_topics = 5

n_top_words = 10

 
 
def top_words_data_frame(model: LatentDirichletAllocation,tf_idf_vectorizer: TfidfVectorizer, n_top_words: int) -> pd.DataFrame:
    rows = []
    feature_names = tf_idf_vectorizer.get_feature_names()
    for topic in model.components_:
        top_words = [feature_names[i]
                     for i in topic.argsort()[:-n_top_words - 1:-1]]
        rows.append(top_words)
    columns = [f'topic {i+1}' for i in range(n_top_words)]
    df = pd.DataFrame(rows, columns=columns)
 
    return df
 
 
def predict_to_data_frame(model: LatentDirichletAllocation, X: np.ndarray) -> pd.DataFrame:
    matrix = model.transform(X)
    columns = [f'P(topic word {i+1})' for i in range(len(model.components_))]
    df = pd.DataFrame(matrix, columns=columns)
    return df
 
 
 
def predict_new_text(text_to_predict_list):
    lda = joblib.load('./Q4/lda-joblib.pkl')
    tf_idf_vectorizer = joblib.load('./Q4/tf_idf_vectorizer.pkl')
    cuts = []
    for text in text_to_predict_list:
        cut_text = [' '.join(jieba.lcut(text))]
        cuts += cut_text
    print(cuts)
    text_tf_idf = tf_idf_vectorizer.transform(cuts).toarray()
    print(text_tf_idf.shape)
    print(text_tf_idf)

    nonzero = np.nonzero(text_tf_idf)
    print(nonzero)
    print(np.array(nonzero).ndim)
    text_predict_df = predict_to_data_frame(lda, text_tf_idf)
    print('预测文本的主题概率分布情况:')
    print(text_predict_df)
if not os.path.exists(source_path):
    print(f'请用浏览器打开 {url} 并下载该文件(如果没有自动下载，则可以在浏览器中按键盘快捷键 ctrl s 来启动下载)')
    os.exit()
df = (
    pd.read_excel(source_path)
    .drop_duplicates()
    .rename(columns={
        document_column_name: 'text'
    }))
# 去重、去缺失、分词
df['cut'] = (
    df['text']
    .apply(lambda x: str(x))
    .apply(lambda x: re.sub(pattern, ' ', x))
    .apply(lambda x: " ".join(jieba.lcut(x)))
)

tf_idf_vectorizer = TfidfVectorizer()
tf_idf = tf_idf_vectorizer.fit_transform(df['cut'])
lda = LatentDirichletAllocation(
    n_components=n_topics,
    max_iter=50,
    learning_method='online',
    learning_offset=50,
    random_state=0)

lda.fit(tf_idf)

top_words_df = top_words_data_frame(lda, tf_idf_vectorizer, n_top_words)

top_words_df.to_csv(top_words_csv_path, encoding='utf-8-sig', index=None)

X = tf_idf.toarray()

predict_df = predict_to_data_frame(lda, X)

predict_df.to_csv(predict_topic_csv_path, encoding='utf-8-sig', index=None)


joblib.dump(lda, 'lda-joblib.pkl')
joblib.dump(tf_idf_vectorizer, 'tf_idf_vectorizer.pkl')

#使用 pyLDAvis 进行可视化
data = pyLDAvis.sklearn.prepare(lda, tf_idf, tf_idf_vectorizer)
pyLDAvis.save_html(data, html_path)
# 清屏
os.system('clear')
# 浏览器打开 html 文件以查看可视化结果
os.system(f'start {html_path}')
print('本次生成了文件：',
      top_words_csv_path,
      predict_topic_csv_path,
      html_path)

print(lda.perplexity(tf_idf))  # 收敛效果

本次生成了文件： ./Q4/top-topic-words.xlsx ./Q4/document-distribution.xlsx ./Q4/document-lda-visualization-top.html
1040.789261446879
